In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES']='1'
from os.path import abspath, dirname, join, basename, isdir
import functools
import json
from addict import Dict
import glob
import yaml
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader, DistributedSampler
from torch.distributed import init_process_group, destroy_process_group
import torch.multiprocessing as mp
import timm
import shutil
import time
from dataset import TotalSegmentatorData
from train import Train

In [2]:
CONFIG_FILE = "config/demo_config.yaml"

In [3]:
cfgs = Dict(yaml.load(open(CONFIG_FILE, "r"), Loader=yaml.Loader))
paths = cfgs.paths
data_cfgs = cfgs.dataset_params
optim_cfgs = cfgs.optimizer_params
schedule_cfgs = cfgs.scheduler_params
model_cfgs = cfgs.model_params
train_cfgs = cfgs.train_params
test_cfgs = cfgs.test_params

## Training

In [4]:
def SaveConfigFile(src, paths):
    results_path = dirname(abspath(paths.model_ckpts_dest))
    model_id = paths.model_ckpts_dest.split(".", 1)[0][-2:]
    filename = basename(src).split(".", 1)[0] + "_" + model_id + ".yaml"
    cp_path = join(results_path, filename)

    if not isdir(results_path):
        os.makedirs(results_path)
    shutil.copy(src, cp_path)
    
def TestDataLoader(cfile):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    cfgs = Dict(yaml.load(open(abspath(cfile), "r"), Loader=yaml.Loader))
    train = Train(cfgs)
    train.LoopDataset()

def main(cfile):
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    torch.manual_seed(42)
    cfgs = Dict(yaml.load(open(abspath(cfile), "r"), Loader=yaml.Loader))
    SaveConfigFile(CONFIG_FILE, cfgs.paths)
    train = Train(cfgs)
    train.RunDDP()

In [5]:
main(CONFIG_FILE)

/home/sgteam/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/sgteam/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/opti

4
0
epoch 1/2 | block: 1, block_size: 1 | loss: 5.25586, dice: -0.003, PFbeta: 0.000, best: 0.000
epoch 1/2 | block: 2, block_size: 1 | loss: 5.33143, dice: -0.003, PFbeta: 0.000, best: 0.000
epoch 1/2 | block: 3, block_size: 1 | loss: 5.41859, dice: -0.002, PFbeta: 0.000, best: 0.000
epoch 1/2 | block: 4, block_size: 1 | loss: 5.48373, dice: -0.002, PFbeta: 0.000, best: 0.000
2
1
3


ProcessRaisedException: 

-- Process 4 terminated with the following error:
Traceback (most recent call last):
  File "/home/sgteam/miniconda3/lib/python3.9/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/isaiah/TotalSegmentatorProj/train.py", line 213, in _TrainModelDDP
    samples = torch.tensor(samples)
ValueError: too many dimensions 'str'


## Testing